In [244]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

### Задание 1.
Обязательная часть  
Будем парсить страницу со свежеми новостям на habr.com/ru/all/.  
Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово.  
Эти слова определяем в начале кода в переменной, например:  
KEYWORDS = ['python', 'парсинг']  

Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы).  
В итоге должен формироваться датафрейм вида  <дата> - <заголовок> - <ссылка>  

In [237]:
KEYWORDS = ['python', 'парсинг']

In [238]:
res = requests.get('http://habr.com/ru/all/')
res_soup = BeautifulSoup(res.text, 'html.parser')

In [239]:
#данные со страницы с постами
page = res_soup.find_all('div', class_='posts_list')

In [240]:
# получим данные по времени поста
date = list(map(lambda x: x.find_all('span', class_='post__time'), page))
#date = res_soup.findAll('span', {'class': 'post__time'}) еще вариант решения
date[0][0].get_text()

'сегодня в 18:05'

In [144]:
#получим данные по заголовку поста
lable = list(map(lambda x: x.find_all('h2', class_='post__title'), page))
lable[0][0].get_text().replace('\n', '')

'Приглашаем на DINS JS EVENING: разбираем Chrome DevTools и Cypress'

In [272]:
#получим ссылку постов
link = list(map(lambda x: x.find('a').get('href'), lable[0]))
link[0]

'https://habr.com/ru/company/dins/blog/537534/'


Дополнительная часть (необязательная)  
Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.  
Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  
Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст_статьи>  

In [416]:
res_text = requests.get('https://habr.com/ru/company/skillfactory/blog/537522/')
res_soup_text = BeautifulSoup(res_text.text, 'html.parser')

In [419]:
#получим данные их блока с текстом
text = res_soup_text.find_all('div', class_='post__text post__text-html post__text_v1')
text[0].get_text()

'Привет, Хабр! Мы уже долгое время следим за динамикой развития IT-профессий на мировом рынке. И сейчас решили сделать топ специальностей, которые будут особенно актуальны в 2021 году. Список составлен на основе роста зарплат на глобальном рынке, востребованности специалистов отрасли и динамики развития профессии в целом. И сразу скажем, в список вошло только 10 профессий. Это не значит, что другие профессии хуже — просто они растут чуть медленнее. Итак, поехали!\n\n\n\nData Analyst\r\nСредняя зарплата в год:\n\n\nЗдесь и ниже — статистика по зарплатам на январь 2021 года предоставлена Glassdoor.\n\r\nАналитик данных решает конкретные вопросы бизнеса с помощью аналитики.\n\r\nПо сравнению с Data Science, о котором мы расскажем далее, у аналитиков зарплата поменьше. Но у Data Analyst порог входа в профессию значительно ниже. Здесь не нужно иметь глубокие знания математики, статистики и теории вероятностей — достаточно просто владеть инструментами аналитика.\xa0\n\r\nРазброс зарплат на р

In [512]:
#не самое изящное решение, но к сожалению не хвататет времени совсем

def get_info_from_hubr(res):
    df = pd.DataFrame(columns=['date', 'lable', 'link', 'text'])
    date_ = []
    lable_ = []
    link_ = []
    text_ = []
    res_soup = BeautifulSoup(res.text, 'html.parser')
    #данные со страницы с постами
    page = res_soup.find_all('div', class_='posts_list')
    
    # получим данные по времени поста
    date = list(map(lambda x: x.find_all('span', class_='post__time'), page))
    for i in date[0]:
        date_.append(i.get_text())
    df['date'] = date_
    
    #получим данные по заголовку поста
    lable = list(map(lambda x: x.find_all('h2', class_='post__title'), page))
    for i in lable[0]:
        lable_.append(i.get_text().replace('\n', ''))
    df['lable'] = lable_
    
    #получим ссылку постов
    link = list(map(lambda x: x.find('a').get('href'), lable[0]))
    for i in link:
        link_.append(i)
    df['link'] = link_
    
    for i in link:
        link_.append(i)
    for i in link:
        res_text = requests.get(i)
        res_soup_text = BeautifulSoup(res_text.text, 'html.parser')
        #получим данные их блока с текстом
        text = res_soup_text.find_all('div', class_='post__text post__text-html post__text_v1')
        if text == []:
            text_.append('Nan')
        elif text != '':
            text = res_soup_text.find_all('div', class_='post__text post__text-html post__text_v1')
            text_.append(text)
    df['text'] = text_
    return df

In [513]:
get_info_from_hubr(res)

,date,lable,link,text
0,сегодня в 18:05,Трюки с виртуальной памятью,https://habr.com/ru/company/otus/blog/537568/,Nan
1,сегодня в 17:18,Разбираем XLNet,https://habr.com/ru/post/536692/,"[<div class=""post__text post__text-html post__..."
2,сегодня в 17:17,Строим надёжную конкурентность с FSP и моделир...,https://habr.com/ru/company/skillfactory/blog/...,"[<div class=""post__text post__text-html post__..."
3,сегодня в 16:59,"Истории основателей: Грейс Гэри, основательниц...",https://habr.com/ru/post/535994/,"[<div class=""post__text post__text-html post__..."
4,сегодня в 16:47,Архитектура и программирование микрокалькулято...,https://habr.com/ru/post/537558/,"[<div class=""post__text post__text-html post__..."
5,сегодня в 16:40,Возможности настройки привилегии и безопасност...,https://habr.com/ru/company/infowatch/blog/537...,"[<div class=""post__text post__text-html post__..."
6,сегодня в 16:39,Создание Dockers в Corel Draw,https://habr.com/ru/post/537556/,Nan
7,сегодня в 16:35,На ком лежит ответственность за качество прогр...,https://habr.com/ru/company/otus/blog/537554/,Nan
8,сегодня в 16:29,Камера iPhone 12 Pro Max: Разбор,https://habr.com/ru/company/droider/blog/531740/,"[<div class=""post__text post__text-html post__..."
9,сегодня в 16:26,"Rust crashcourse. Правило трёх — параметры, ит...",https://habr.com/ru/post/537546/,"[<div class=""post__text post__text-html post__..."
